In [ ]:
!pip install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install chromadb
!pip install whoosh
!pip install llama-cpp-python


In [24]:
import zipfile
import os
import json
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import files
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from whoosh import index
from whoosh.fields import Schema, TEXT
from whoosh.writing import AsyncWriter
from whoosh.qparser import QueryParser
from whoosh.index import create_in

In [3]:
uploaded = files.upload()

archive_name = 'wiki_html.zip'

with zipfile.ZipFile(archive_name, 'r') as zip_ref:
    zip_ref.extractall('wiki_html')

print("Файлы в папке wiki_html:")
print(os.listdir('wiki_html'))


Saving wiki_html.zip to wiki_html.zip
Файлы в папке wiki_html:
['Ньютон_Исаак.html', 'Лагранж_Жозеф_Луи.html', 'Риман_Бернхард.html', 'Паскаль_Блез.html', 'Кантор_Георг.html', 'ДАламбер_Жан_Лерон.html', 'Лейбниц_Готфрид_Вильгельм.html', 'Бернулли_Иоганн.html', 'Якоби_Карл_Густав_Якоб.html', 'Гамильтон_Уильям.html', 'Абель_Нильс_Хенрик.html', 'Эйлер_Леонард.html', 'Галуа_Эварист.html', 'Гильберт_Давид.html', 'Лаплас_Пьер_Симон.html', 'Вейерштрасс_Карл.html', 'Декарт_Рене.html', 'Коши_Огюстен_Луи.html', 'Ферма_Пьер.html', 'Гаусс_Карл_Фридрих.html']


In [4]:
def load_and_clean_wikipedia_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'lxml')

    for tag in soup(['script', 'style', 'nav', 'footer', 'header']):
        tag.decompose()

    for edit_tag in soup.select('.mw-editsection, .mw-parser-output sup.reference, .navbox'):
        edit_tag.decompose()

    for infobox in soup.select('.infobox, .toc, .sistersitebox, .catlinks, #footer, .hatnote, .metadata'):
        infobox.decompose()

    content = soup.select_one('div.mw-parser-output').get_text(separator=' ', strip=True)

    return content

html_dir = 'wiki_html'
html_files = [os.path.join(html_dir, file) for file in os.listdir(html_dir) if file.endswith('.html')]



all_documents = []
for html_file in html_files:
    cleaned_text = load_and_clean_wikipedia_html(html_file)
    all_documents.append(Document(page_content=cleaned_text))



In [5]:
!ls wiki_html

Абель_Нильс_Хенрик.html  ДАламбер_Жан_Лерон.html	 Ньютон_Исаак.html
Бернулли_Иоганн.html	 Декарт_Рене.html		 Паскаль_Блез.html
Вейерштрасс_Карл.html	 Кантор_Георг.html		 Риман_Бернхард.html
Галуа_Эварист.html	 Коши_Огюстен_Луи.html		 Ферма_Пьер.html
Гамильтон_Уильям.html	 Лагранж_Жозеф_Луи.html		 Эйлер_Леонард.html
Гаусс_Карл_Фридрих.html  Лаплас_Пьер_Симон.html		 Якоби_Карл_Густав_Якоб.html
Гильберт_Давид.html	 Лейбниц_Готфрид_Вильгельм.html


In [ ]:
all_documents[5]

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
split_docs = text_splitter.split_documents(all_documents)


for idx, doc in enumerate(split_docs):
    print(f"Чанк {idx+1}:")
    print(doc.page_content)
    print("-" * 50)


Чанк 1:
Не следует путать с современником-генералом Жозефом Лагранжем . Жозе́ф Луи́ Лагра́нж ( фр. Joseph Louis Lagrange , итал. Giuseppe Lodovico Lagrangia ; 25 января 1736 , Турин — 10 апреля 1813 , Париж ) — французский математик , астроном и механик итальянского происхождения. Наряду с Эйлером — крупнейший математик XVIII века . Особенно прославился исключительным мастерством в области обобщения и синтеза накопленного научного материала. Автор классического трактата « Аналитическая механика », в котором установил фундаментальный « принцип возможных перемещений » и завершил математизацию механики .
--------------------------------------------------
Чанк 2:
установил фундаментальный « принцип возможных перемещений » и завершил математизацию механики . Внёс огромный вклад в математический анализ , теорию чисел , в теорию вероятностей и численные методы , создал вариационное исчисление . Член Прусской академии наук (1766—1787; иностранный член в период 1756—1766 и с 1787 года) , Парижс

In [8]:
# Создание векторного индекса ChromaDB для поиска
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = Chroma.from_documents(split_docs, embeddings, collection_metadata={"hnsw:space": "cosine"})



<ipython-input-8-843d0439fc36>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Fa

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
user_query = "теорема"
query_embedding = embeddings.embed_query(user_query)
top_n = 8

In [10]:

results = vectorstore.similarity_search_with_relevance_scores(user_query, k = top_n)


In [11]:
for idx, result in enumerate(results):
    document = result[0]
    score = result[1]
    print(f"Релевантный документ {idx+1}:")
    print("Score:", score)
    print(document.page_content)
    print("-" * 50)


Релевантный документ 1:
Score: 0.6956310868263245
вручаемая Немецким математическим обществом ; Нумерующая функция Кантора — отображение декартовой степени множества натуральных чисел в само себя; Теорема Кантора о том, что мощность множества всех подмножеств данного множества строго больше мощности самого множества; Теорема Кантора — Бендиксона Теорема Кантора — Бернштейна о равномощности множеств A и B при условии равномощности A подмножеству B и равномощности B подмножеству A ; Теорема Кантора — Гейне о равномерной непрерывности непрерывной функции на компакте ; Функция Кантора (Канторова лестница). Сочинения Cantor G. Gesammelte
--------------------------------------------------
Релевантный документ 2:
Score: 0.6580475568771362
Паскаля Признак Паскаля Суммирующая машина Паскаля Теорема Паскаля Сочинения Блеза Паскаля Опыт о конических сечениях (Essai pour les coniques, 1639) — теорема Паскаля о том, что во всяком шестиугольнике, вписанном в эллипс, гиперболу или параболу, точки пер

In [12]:
# индекс для лексического поиска

schema = Schema(content=TEXT(stored=True))

index_dir = "indexdir_new"
if not os.path.exists(index_dir):
    os.mkdir(index_dir)


ix = create_in(index_dir, schema)


with ix.writer() as writer:
    for doc in split_docs:
        writer.add_document(content=doc.page_content)

with ix.searcher() as searcher:
    print(f"Количество документов в индексе: {searcher.doc_count_all()}")

Количество документов в индексе: 555


In [13]:

with ix.searcher() as searcher:
    query_str = "теорема"
    query = QueryParser("content", ix.schema).parse(query_str)
    results = searcher.search(query, limit=8)

    if results:
        for i, result in enumerate(results):
            print(f"Релевантный документ {i+1}:\n{result['content']}\n{'-'*50}")
    else:
        print("Ничего не найдено.")


Релевантный документ 1:
Bd. 4 Berlin, 1902 «Vorl. ueber Variationsrechnung» // Math. Werke. Bd. 6 Berlin, 1927 Digitalized versions of Weierstraß' original publications are freely available online from the library of the Berlin Brandenburgische Akademie der Wissenschaften . См. также Теорема Вейерштрасса о функции, непрерывной на компакте Теорема Вейерштрасса о целых функциях Аппроксимационная теорема Вейерштрасса Теорема Больцано — Вейерштрасса Теорема Линдемана — Вейерштрасса Теорема Сохоцкого — Вейерштрасса Примечания ↑ Deutsche Nationalbibliothek Record #11876618X // Gemeinsame Normdatei (нем.) — 2012—2016. ↑ 1
--------------------------------------------------
Релевантный документ 2:
вручаемая Немецким математическим обществом ; Нумерующая функция Кантора — отображение декартовой степени множества натуральных чисел в само себя; Теорема Кантора о том, что мощность множества всех подмножеств данного множества строго больше мощности самого множества; Теорема Кантора — Бендиксона Теор

In [14]:
from llama_cpp import Llama


In [15]:
USER_QUERY = "У какого математика в теоремах были ошибки, нечеткие и размытые формулировки?"
top_n = 4
GROUNDED_SYSTEM_PROMPT = "Your task is to answer the user's questions using only the information from the provided documents. Give two answers to each question: one with a list of relevant document identifiers and the second with the answer to the question itself, using documents with these identifiers."


In [ ]:

llm = Llama.from_pretrained(
	repo_id="VlSav/Vikhr-Nemo-12B-Instruct-R-21-09-24-Q4_K_M-GGUF",
	filename="vikhr-nemo-12b-instruct-r-21-09-24-q4_k_m.gguf",
  n_ctx=2048
)



In [17]:
# переформулировка вопроса на несколько более развернутых и понятных

rephrased = llm.create_chat_completion(
      messages = [
          {"role": "system", "content": "Your task is to take the user's question and rephrase it as multiple distinct, contextually expanded sentences that make the question clearer and easier to understand. Each sentence should be concise, focused on specific aspects of the question, and suitable for accurate vector-based search. Avoid adding unnecessary details or assumptions, and ensure that each rephrased sentence can stand alone as a relevant search query."},
          {
              "role": "user",
              "content": USER_QUERY
          }
      ]
)

rephrased = rephrased['choices'][0]['message']['content']
print(rephrased)

llama_perf_context_print:        load time =   72588.89 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   117 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    37 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  110265.82 ms /   154 tokens


Кто из математиков известен тем, что в его теоремах встречались ошибки?
Какие математические работы содержали нечеткие или размытые формулировки?


In [18]:
# создание 2-3 ключевых слов для лексического поиска

rephrased_2 = llm.create_chat_completion(
      messages = [
          {"role": "system", "content": "Your task is to extract the 2-3 most relevant keywords from the user's question. Select words that best capture the core meaning and intent of the question. Avoid generic terms or overly broad words; focus on those that are specific enough to improve accuracy in search or retrieval tasks. Present only the keywords, separated by spaces."},
          {
              "role": "user",
              "content": USER_QUERY
          }
      ]
)

rephrased_2 = rephrased_2['choices'][0]['message']['content']
print(rephrased_2)

Llama.generate: 9 prefix-match hit, remaining 96 prompt tokens to eval
llama_perf_context_print:        load time =   72588.89 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    96 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     9 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   70970.77 ms /   105 tokens


математик ошибки формулировки


In [19]:
# поиск векторный
results = vectorstore.similarity_search_with_relevance_scores(rephrased, k = top_n)

for idx, result in enumerate(results):
    document = result[0]
    score = result[1]
    print(f"Релевантный документ {idx+1}:")
    print("Score:", score)
    print(document.page_content)
    print("-" * 50)


Релевантный документ 1:
Score: 0.7929834127426147
без какого-либо определения. Отсутствовала полная теория сходимости. Как следствие, многие теоремы содержали ошибки, нечёткие или чрезмерно широкие формулировки. Наглядный образ «дикой» функции Вейерштрасса Вейерштрасс завершил построение фундамента математического анализа , прояснил тёмные места, построил ряд доказательных контрпримеров (аномальных функций), например, всюду непрерывную, но нигде не дифференцируемую функцию. Он сформулировал логическое обоснование анализа на основе построенной им теории действительных (вещественных) чисел и так называемого ε-δ-языка. Например, он строго
--------------------------------------------------
Релевантный документ 2:
Score: 0.7730021476745605
три нерешённые проблемы, которые… действительно сыграли важную роль в развитии математики на протяжении последующих сорока с лишним лет. Любой математик, решивший одну из них, занимал почётное место в математическом сообществе. Мы, математики, часто оцени

In [21]:
# поиск лексический
with ix.searcher() as searcher:
    query_str = rephrased_2
    query = QueryParser("content", ix.schema).parse(query_str)
    results_2 = searcher.search(query, limit=8)

    if results_2:
        for i, result in enumerate(results_2):
            print(f"Релевантный документ {i+1}:\n{result['content']}\n{'-'*50}")
    else:
        print("Ничего не найдено.")

Ничего не найдено.


In [22]:
# убираем дупликаты
unique_documents = {}


for idx, result in enumerate(results):
    document = result[0]
    score = result[1]
    content = document.page_content

    # Уникальный ключ для каждого документа на основе его контента
    unique_key = content[:50]  # можно использовать первые 50 символов
    if unique_key not in unique_documents:
        unique_documents[unique_key] = {
            "doc_id": len(unique_documents),
            "title": "",  # оставляем название пустым
            "content": content
        }


with ix.searcher() as searcher:
    query_str = rephrased_2
    query = QueryParser("content", ix.schema).parse(query_str)
    results_2 = searcher.search(query, limit=8)

    for result in results_2:
        content = result['content']


        unique_key = content[:50]
        if unique_key not in unique_documents:
            unique_documents[unique_key] = {
                "doc_id": len(unique_documents),
                "title": "",  # оставляем название пустым
                "content": content
            }

documents = list(unique_documents.values())

# тест
for doc in documents[:5]:
    print(doc)

{'doc_id': 0, 'title': '', 'content': 'без какого-либо определения. Отсутствовала полная теория сходимости. Как следствие, многие теоремы содержали ошибки, нечёткие или чрезмерно широкие формулировки. Наглядный образ «дикой» функции Вейерштрасса Вейерштрасс завершил построение фундамента математического анализа , прояснил тёмные места, построил ряд доказательных контрпримеров (аномальных функций), например, всюду непрерывную, но нигде не дифференцируемую функцию. Он сформулировал логическое обоснование анализа на основе построенной им теории действительных (вещественных) чисел и так называемого ε-δ-языка. Например, он строго'}
{'doc_id': 1, 'title': '', 'content': 'три нерешённые проблемы, которые… действительно сыграли важную роль в развитии математики на протяжении последующих сорока с лишним лет. Любой математик, решивший одну из них, занимал почётное место в математическом сообществе. Мы, математики, часто оцениваем свои успехи мерой того, какие из гильбертовых проблем удалось ещё 

In [25]:
# далее адаптировано из карточки Vikhr-Nemo-12B-Instruct-R-21-09-24 на HF
sample_history = [
    {'role': 'system', 'content': GROUNDED_SYSTEM_PROMPT},
    {'role': 'documents', 'content': json.dumps(documents, ensure_ascii=False)},
    {'role': 'user', 'content': rephrased}
]

In [31]:

relevant_indexes = llm.create_chat_completion(
    messages=sample_history,
    temperature=0.0,
)
print('Using documents: ' + relevant_indexes['choices'][0]['message']['content'] + '\n----')

final_answer = llm.create_chat_completion(
    messages=sample_history + [{'role': 'assistant', 'content': relevant_indexes}],
    temperature=0.3,
)

print(final_answer['choices'][0]['message']['content'])


Llama.generate: 866 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =  547782.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    10 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   10858.20 ms /    11 tokens
Llama.generate: 867 prefix-match hit, remaining 261 prompt tokens to eval


Using documents: {"relevant_doc_ids": [0]}
----


llama_perf_context_print:        load time =  547782.82 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   261 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   458 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  665027.23 ms /   719 tokens


В историческом контексте математических наук, многие теории и теоремы, особенно на ранних этапах развития математики, могли содержать ошибки, нечеткие или чрезмерно широкие формулировки. Это было обусловлено отсутствием полной теории сходимости и непониманием некоторых основных концепций. В качестве примера можно привести трудности, с которыми столкнулся математический анализ до того, как были уточнены основные понятия, такие как непрерывность и дифференцируемость функций.

Один из математиков, который сыграл важную роль в прояснении этих вопросов, – это Герман Вейерштрасс. Он смог завершить построение фундамента математического анализа, прояснить "тёмные места" и построил ряд доказательных контрпримеров. Среди таких контрпримеров – функция, которая всюду непрерывна, но нигде не дифференцируема. Это была значительная теорема в разработке математического представления действительных чисел и их характеристик, известных как «ε-δ-язык».

Неоднозначности и критические замечания, связанные с